## Webscrapping from metroscubicos.com

In [1]:
# import instances and libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import re
import pandas as pd
# To show a progess bar in loops:
from tqdm import tqdm.notebook.tqdm as tqdm
import csv

In [9]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been 

In [2]:
# List all entities available in metroscubicos.com

entidades = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima',
            'Distrito Federal', 'Durango', 'Estado De México', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán',
             'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí',
             'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']

In [3]:
# make smallcaps and change spaces for dashes, and select which entities
entidadesLower = [x.lower().replace(' ', '-') for x in entidades]

In [4]:
entidadesLower

['aguascalientes',
 'baja-california',
 'baja-california-sur',
 'campeche',
 'chiapas',
 'chihuahua',
 'coahuila',
 'colima',
 'distrito-federal',
 'durango',
 'estado-de-méxico',
 'guanajuato',
 'guerrero',
 'hidalgo',
 'jalisco',
 'michoacán',
 'morelos',
 'nayarit',
 'nuevo-león',
 'oaxaca',
 'puebla',
 'querétaro',
 'quintana-roo',
 'san-luis-potosí',
 'sinaloa',
 'sonora',
 'tabasco',
 'tamaulipas',
 'tlaxcala',
 'veracruz',
 'yucatán',
 'zacatecas']

In [5]:
# Initialize variables
cityList = []
listLinks = []
tipos = ['casas', 'departamentos', 'terrenos']
baseUrl = 'https://inmuebles.metroscubicos.com/'
csvPath = './Resources/'

### Script para hacer el webscrapping por ciudad

In [15]:
## Read file by city
ciudad = entidadesLower[0]
csvfile = csvPath+ciudad+".csv"
data= pd.read_csv(csvfile)
data.head()

,ID,entidad,tipo,link
0,1425548080,aguascalientes,venta,https://casa.metroscubicos.com/MLM-1425548080-...
1,1397663987,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1397663987-...
2,1398422703,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1398422703-...
3,1413775652,aguascalientes,bonita,https://casa.metroscubicos.com/MLM-1413775652-...
4,1411133552,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1411133552-...


In [16]:
# Select which city
entidadesLower[0:1]

In [22]:
for ciudad in tqdm(entidadesLower):
    # Read csv file and get links
    csvfile = csvPath+ciudad+".csv"
    data= pd.read_csv(csvfile)
    links = data["link"].tolist()
    
    # Loop through all links and webscrape info. Info is stores in a dictionary
    for link in tqdm(links):
        # Clean dictionary for each link
        dict = {}
        dict["link"] = link
        try: 
            asset_soup = soup(requests.get(link).content, 'html.parser')
        except:
            print(f"Cannot retrieve {link} ... passing")
            continue
        # Get price and currency symbol
        if asset_soup.find('span', class_="andes-money-amount__currency-symbol") != None:
            simbolo = asset_soup.find('span', class_="andes-money-amount__currency-symbol").get_text()
        else:
            simbolo = ""
        if asset_soup.find('span', class_="andes-money-amount__fraction") != None:
            monto = asset_soup.find('span', class_="andes-money-amount__fraction").get_text()
        else:
            monto = ""
        precio = simbolo+monto
        dict["precio"] = precio
        if asset_soup.find('ol', class_="andes-breadcrumb") != None:
            divBreadcrumbs = asset_soup.find('ol', class_="andes-breadcrumb")
            aBreadcrumbs = divBreadcrumbs.find_all('a')
            dict['colonia'] = aBreadcrumbs[-1].get_text()
            dict['municipio'] = aBreadcrumbs[-2].get_text()
        # Find table and process columns and rows
        if asset_soup.find('tbody', class_="andes-table__body") == None:
            tabla = ""
            pass
        else:
            tabla = asset_soup.find('tbody', class_="andes-table__body")
            tablaDatos = asset_soup.find('table', class_="andes-table") 
            nombreDatos = tablaDatos.find_all('th')
            dataDatos = tablaDatos.find_all('td')
            for i in range(len(nombreDatos)):
                dict[nombreDatos[i].get_text()] = dataDatos[i].get_text()
        
        # Add dictionary values to global list
        if dict not in cityList:
            cityList.append(dict)

results_df = pd.DataFrame(cityList)
final_df = pd.merge(data, results_df, on="link")
file = csvPath+city+"-webscrapeResults.csv"
final_df.to_csv(file, index=False)
            

C:\Users\CAC\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1908 [00:00<?, ?it/s]

{'link': 'https://terreno.metroscubicos.com/MLM-1406187667-vendo-terreno-en-fraccionamiento-las-plazas-_JM',
 'precio': '$187,500,000',
 'colonia': 'Las Plazuelas',
 'municipio': 'Aguascalientes',
 'Superficie total': '250 ha',
 'Superficie construida': '90 m²',
 'Ambientes': '3',
 'Recámaras': '2',
 'Baños': '1',
 'Estacionamientos': '1',
 'Bodegas': '1',
 'Cantidad de pisos': '3',
 'Orientación': 'Norte',
 'Antigüedad': '0 años',
 'Cuota mensual de mantenimiento': '0 MXN',
 'RecÃ¡maras': '2',
 'BaÃ±os': '2',
 'AntigÃ¼edad': '1 aÃ±os',
 'Tipo de casa': 'Casa',
 'Tipo de propiedad': 'Casa en condominio',
 'OrientaciÃ³n': 'Norte',
 'Departamentos por piso': '4',
 'Número de piso de la unidad': '3',
 'Disposición': 'Contrafrente',
 'NÃºmero de piso de la unidad': '2',
 'Acceso': 'Otro',
 'Metros de frente': '45 m',
 'Metros de fondo': '55 m',
 'Forma del terreno': 'Plano',
 'DisposiciÃ³n del lote': 'Perimetral'}

### Multithreading

In [6]:
from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count

In [7]:
def parseWeb(url):
    dict = {}
    try: 
        asset_soup = soup(requests.get(url).content, 'html.parser')
    except:
        print(f"Cannot retrieve {url} ... passing")
        return dict
    # Get price and currency symbol
    if asset_soup.find('span', class_="andes-money-amount__currency-symbol") != None:
        simbolo = asset_soup.find('span', class_="andes-money-amount__currency-symbol").get_text()
    else:
        simbolo = ""
    if asset_soup.find('span', class_="andes-money-amount__fraction") != None:
        monto = asset_soup.find('span', class_="andes-money-amount__fraction").get_text()
    else:
        monto = ""
    precio = simbolo+monto
    dict["precio"] = precio
    if asset_soup.find('ol', class_="andes-breadcrumb") != None:
        divBreadcrumbs = asset_soup.find('ol', class_="andes-breadcrumb")
        aBreadcrumbs = divBreadcrumbs.find_all('a')
        dict['colonia'] = aBreadcrumbs[-1].get_text()
        dict['municipio'] = aBreadcrumbs[-2].get_text()
    # Find table and process columns and rows
    if asset_soup.find('tbody', class_="andes-table__body") == None:
        tabla = ""
        pass
    else:
        tabla = asset_soup.find('tbody', class_="andes-table__body")
        tablaDatos = asset_soup.find('table', class_="andes-table") 
        nombreDatos = tablaDatos.find_all('th')
        dataDatos = tablaDatos.find_all('td')
        for i in range(len(nombreDatos)):
            dict[nombreDatos[i].get_text()] = dataDatos[i].get_text()
    #if dict not in cityList:
    #    cityList.append(dict)
    return dict

In [ ]:
if __name__ == '__main__':
    ciudad = entidadesLower[0]
    csvfile = csvPath+ciudad+".csv"
    data= pd.read_csv(csvfile)
    links = data["link"].tolist()
    pool = Pool(cpu_count() * 200)
    results = pool.map(parseWeb, links)
    if results not in cityList and len(results) != 0:
        cityList.append(results)
        print(f"Processed {len(results)} links")

Cannot retrieve https://casa.metroscubicos.com/MLM-1419146480-casa-en-venta-arboledas-de-paso-blanco-aguascalientes-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-951074747-casa-en-xarama-residencial-girona-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-936352294-casa-sola-en-venta-jardines-de-la-asuncion-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1332709573-go-liquidacion-hipotecaria-de-casa-hacienda-tetelpa-ags-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1362085554-casa-sola-residencial-en-venta-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-953514589-casa-en-venta-en-eden-los-sabinos-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1401341367-casa-fraccionamiento-ladrilleras-los-arellano-_JM ... passing
Cannot retrieve https://casa.metroscubicos.com/MLM-1401330560-casa-en-venta-con-locales-comerciales-en-fracc-ejido-cumbres-iii-cerca-de-linea-verde-_JM ... pas